### 2. (5 pts) Binary classification with logistic regression with over sampling

Applying oversampling to balance the dataset and training and testing the model with the balanced dataset.And performing the same steps which are performed on the imbalanced dataset.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
heart_disease_df_1 = pd.read_csv('/content/heart_disease_train.csv')
heart_disease_df_original = heart_disease_df_1.copy()
heart_disease_df_original.head()

,gender,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,63,1.0,0,0.0,0.0,0,1,0,190.0,148.0,90.0,27.13,72.0,86.0,0
1,1,45,3.0,0,0.0,0.0,0,0,0,162.0,125.0,89.0,27.98,85.0,83.0,0
2,0,51,1.0,1,2.0,0.0,0,0,0,261.0,127.0,81.0,20.24,75.0,96.0,0
3,1,45,3.0,1,30.0,0.0,0,0,0,250.0,126.0,89.5,28.68,75.0,92.0,0
4,0,45,2.0,1,3.0,0.0,0,0,0,250.0,130.0,80.0,20.24,90.0,86.0,0


In [2]:
heart_disease_df = heart_disease_df_original.copy()

In [3]:
heart_disease_df['TenYearCHD'].value_counts()

,count
TenYearCHD,
0,2874
1,516


In [4]:
zero_class = heart_disease_df[heart_disease_df['TenYearCHD'] == 0]
one_class = heart_disease_df[heart_disease_df['TenYearCHD'] == 1]
zero_majority = len(zero_class)
one_minority = len(one_class)
samples_needed = zero_majority - one_minority
choosing = np.random.choice(one_class.index, size=samples_needed, replace=True)
oversampled_minority = one_class.loc[choosing]
balanced_heart_disease_df = pd.concat([zero_class, one_class, oversampled_minority])
balanced_heart_disease_df = balanced_heart_disease_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [5]:
heart_disease_df = balanced_heart_disease_df.copy()

In [6]:
heart_disease_df.shape

(5748, 16)

In [7]:
heart_disease_df['TenYearCHD'].value_counts()

,count
TenYearCHD,
0,2874
1,2874


In [8]:
heart_disease_df['cigsPerDay'] = heart_disease_df['cigsPerDay'].fillna(heart_disease_df.groupby('currentSmoker')['cigsPerDay'].transform('median'))

In [9]:
heart_disease_df['glucose'] = heart_disease_df['glucose'].fillna(heart_disease_df.groupby('diabetes')['glucose'].transform('median'))

In [10]:
heart_disease_df['BMI'] = heart_disease_df['BMI'].fillna(heart_disease_df.groupby(['gender', 'age'])['BMI'].transform('median'))

In [11]:
heart_disease_df['BPMeds'] = heart_disease_df['BPMeds'].fillna(heart_disease_df['BPMeds'].median())

In [12]:
heart_disease_df['education'] = heart_disease_df['education'].fillna(heart_disease_df['education'].median())

In [13]:
heart_disease_df['totChol'] = heart_disease_df['totChol'].fillna(heart_disease_df['totChol'].median())

In [14]:
heart_disease_df['heartRate'] = heart_disease_df['heartRate'].fillna(heart_disease_df['heartRate'].median())

In [15]:
heart_disease_df.isnull().sum()

,0
gender,0
age,0
education,0
currentSmoker,0
cigsPerDay,0
BPMeds,0
prevalentStroke,0
prevalentHyp,0
diabetes,0
totChol,0


In [16]:
logarthmic_columns = ['cigsPerDay', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']
for i in logarthmic_columns:
    heart_disease_df[i] = np.log1p(heart_disease_df[i])
heart_disease_df.head()

,gender,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,1.0,1,3.044522,0.0,0,0,0,5.231109,4.844187,4.219508,3.137232,4.158883,4.290459,0
1,1,67,2.0,1,4.110874,0.0,0,1,0,5.568345,5.141664,4.615121,3.165897,4.290459,4.382027,1
2,1,52,2.0,0,0.000000,0.0,1,0,0,5.313206,4.919981,4.553877,3.431727,4.430817,4.219508,1
3,0,63,2.0,1,3.713572,0.0,0,0,0,5.192957,4.762174,4.248495,3.141995,4.564348,4.330733,1
4,0,51,1.0,1,1.386294,0.0,0,0,0,5.509388,4.722953,4.317488,3.261552,4.369448,4.369448,0


In [17]:
coloumns_norm = heart_disease_df.drop(['TenYearCHD'], axis=1).columns
normalized_df = heart_disease_df.copy()
for i in coloumns_norm:
    mean_c = heart_disease_df[i].mean()
    std_c = heart_disease_df[i].std()
    normalized_df[i] = (heart_disease_df[i] - mean_c) / std_c
normalized_df.head()

,gender,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1.049136,-1.415308,-0.919989,1.001654,1.087111,-0.210982,-0.112618,-0.777551,-0.201354,-1.220798,-0.373833,-1.446885,-1.004952,-1.164856,-0.519219,0
1,1.049136,1.802908,0.057297,1.001654,1.798971,-0.210982,-0.112618,1.285866,-0.201354,0.523797,1.354435,1.201818,-0.813156,-0.290919,-0.117373,1
2,1.049136,0.078864,0.057297,-0.998175,-0.945308,-0.210982,8.878041,-0.777551,-0.201354,-0.796091,0.066512,0.791780,0.965493,0.641343,-0.830593,1
3,-0.953000,1.343163,0.057297,1.001654,1.533746,-0.210982,-0.112618,-0.777551,-0.201354,-1.418166,-0.850309,-1.252808,-0.973083,1.528266,-0.342475,1
4,-0.953000,-0.036073,-0.919989,1.001654,-0.019865,-0.210982,-0.112618,-0.777551,-0.201354,0.218804,-1.078172,-0.790888,-0.173133,0.233727,-0.172576,0


In [18]:
features = normalized_df.drop('TenYearCHD', axis=1).values
target = normalized_df['TenYearCHD'].values.reshape(-1, 1)

In [19]:
n_smaples, n_features = features.shape
weights = np.zeros((n_features, 1))
bias = 0
learning_rate = 0.01
n_epochs = 10

In [20]:
def sigmoid_function(z):
    return 1 / (1 + np.exp(-z))

In [21]:
def BCE_function(features, target, weights, bias):
    m = features.shape[0]
    linear_comb = np.dot(features, weights) + bias
    y_cap = sigmoid_function(linear_comb)
    cost = -(1 / m) * np.sum(target * np.log(y_cap) + (1 - target) * np.log(1 - y_cap))
    return cost

In [22]:
def gradient_descent(features, target, weights, bias, learning_rate, n_epochs):
    m = features.shape[0]
    loss = []
    for i in range(n_epochs):
        linear_comb = np.dot(features, weights) + bias
        y_cap = sigmoid_function(linear_comb)
        dweight = (1 / m) * np.dot(features.T, (y_cap - target))
        dbias = (1 / m) * np.sum(y_cap - target)
        weights = weights - learning_rate*dweight
        bias = bias-learning_rate*dbias
        cost_calculation = BCE_function(features, target, weights, bias)
        loss.append(cost_calculation)
        print(f"Epoch {i + 1}/{n_epochs}, Cost: {cost_calculation:.4f}")
    return weights, bias, loss

In [23]:
weights, bias, loss = gradient_descent(features, target, weights, bias, learning_rate, n_epochs)

Epoch 1/10, Cost: 0.6924
Epoch 2/10, Cost: 0.6916
Epoch 3/10, Cost: 0.6909
Epoch 4/10, Cost: 0.6901
Epoch 5/10, Cost: 0.6894
Epoch 6/10, Cost: 0.6887
Epoch 7/10, Cost: 0.6880
Epoch 8/10, Cost: 0.6873
Epoch 9/10, Cost: 0.6866
Epoch 10/10, Cost: 0.6859


In [24]:
def predict_function(features, weights, bias):
    linear_comb = np.dot(features, weights) + bias
    y_cap = sigmoid_function(linear_comb)
    return (y_cap >= 0.5).astype(int)

In [25]:
def cal_metrics(y_ground, y_predict):
    truepositive = np.sum((y_ground == 1) & (y_predict == 1))
    truenegative = np.sum((y_ground == 0) & (y_predict == 0))
    falsepositive = np.sum((y_ground == 0) & (y_predict == 1))
    falsenegative = np.sum((y_ground == 1) & (y_predict == 0))
    if (truepositive + falsepositive) > 0:
      precision = truepositive / (truepositive + falsepositive)
    else:
      precision = 0
    accuracy = (truepositive + truenegative) / len(y_ground)
    if (truepositive + falsenegative) > 0:
      recall = truepositive / (truepositive + falsenegative)
    else:
      recall = 0
    return accuracy, precision, recall, truepositive, truenegative, falsepositive, falsenegative
y_predict = predict_function(features, weights, bias)
accuracy, precision, recall, truepositive, truenegative, falsepositive, falsenegative = cal_metrics(target, y_predict)
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"Confusion Matrix: TruePositive={truepositive}, TrueNegative={truenegative}, FalsePositive={falsepositive}, FalseNegative={falsenegative}")

Accuracy: 0.634
Precision: 0.648
Recall: 0.585
Confusion Matrix: TruePositive=1681, TrueNegative=1961, FalsePositive=913, FalseNegative=1193


In [26]:
heart_disease_test_df = pd.read_csv('/content/heart_disease_test.csv')

In [27]:
heart_disease_test_df['cigsPerDay'] = heart_disease_test_df['cigsPerDay'].fillna(heart_disease_test_df.groupby('currentSmoker')['cigsPerDay'].transform('median'))

In [28]:
heart_disease_test_df['glucose'] = heart_disease_test_df['glucose'].fillna(heart_disease_test_df.groupby('diabetes')['glucose'].transform('median'))

In [29]:
heart_disease_test_df['BMI'] = heart_disease_test_df['BMI'].fillna(heart_disease_test_df.groupby(['gender', 'age'])['BMI'].transform('median'))

In [30]:
heart_disease_test_df['BPMeds'] = heart_disease_test_df['BPMeds'].fillna(heart_disease_test_df['BPMeds'].median())

In [31]:
heart_disease_test_df['education'] = heart_disease_test_df['education'].fillna(heart_disease_test_df['education'].median())

In [32]:
heart_disease_test_df['totChol'] = heart_disease_test_df['totChol'].fillna(heart_disease_test_df['totChol'].median())

In [33]:
logarthmic_columns = ['cigsPerDay', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']
for i in logarthmic_columns:
    heart_disease_test_df[i] = np.log1p(heart_disease_test_df[i])
heart_disease_test_df.head()

,gender,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,0,56,1.0,0,0.000000,0.0,0,1,1,5.613128,4.919981,4.394449,3.357942,4.510860,5.351858,1
1,0,46,2.0,0,0.000000,0.0,0,0,0,5.525453,4.804021,4.406719,3.392157,4.564348,4.343805,0
2,0,52,1.0,0,0.000000,0.0,0,0,0,5.620401,4.727388,4.276666,3.283914,4.394449,4.356709,0
3,0,46,3.0,1,3.178054,0.0,0,0,0,5.655992,4.875197,4.442651,3.182212,4.454347,4.454347,0
4,1,69,1.0,1,0.693147,0.0,0,0,0,5.505332,4.820282,4.356709,3.317091,4.262680,4.406719,1


In [34]:
coloumns_norm_test = heart_disease_test_df.drop(['TenYearCHD'], axis=1).columns
normalized_test_df = heart_disease_test_df.copy()
for i in coloumns_norm:
    mean_c = heart_disease_test_df[i].mean()
    std_c = heart_disease_test_df[i].std()
    normalized_test_df[i] = (heart_disease_test_df[i] - mean_c) / std_c
normalized_test_df.head()

,gender,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,-0.835807,0.775315,-0.975564,-1.047718,-0.974490,-0.188062,-0.10351,1.379081,5.734211,0.915297,0.209581,-0.233576,0.536595,1.120294,4.789655,1
1,-0.835807,-0.411663,0.029634,-1.047718,-0.974490,-0.188062,-0.10351,-0.724266,-0.174186,0.405871,-0.531071,-0.144713,0.767522,1.461396,-0.217137,0
2,-0.835807,0.300524,-0.975564,-1.047718,-0.974490,-0.188062,-0.10351,-0.724266,-0.174186,0.957554,-1.020538,-1.086591,0.036958,0.377935,-0.153048,0
3,-0.835807,-0.411663,1.034832,0.953329,1.206125,-0.188062,-0.10351,-0.724266,-0.174186,1.164351,-0.076458,0.115516,-0.649464,0.759910,0.331902,0
4,1.195037,2.318386,-0.975564,0.953329,-0.498889,-0.188062,-0.10351,-0.724266,-0.174186,0.288957,-0.427213,-0.506901,0.260878,-0.462370,0.095343,1


In [35]:
f = normalized_test_df.drop('TenYearCHD', axis=1).values
t = normalized_test_df['TenYearCHD'].values.reshape(-1, 1)
t_pred = predict_function(f, weights, bias)
accuracy, precision, recall, truepositive, truenegative, falsepositive, falsenegative = cal_metrics(t, t_pred)
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"Confusion Matrix: TruePositive={truepositive}, TrueNegative={truenegative}, FalsePositive={falsepositive}, FalseNegative={falsenegative}")

Accuracy: 0.608
Precision: 0.241
Recall: 0.742
Confusion Matrix: TruePositive=95, TrueNegative=421, FalsePositive=299, FalseNegative=33
